In [1]:
import os
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

In [4]:
loader = TextLoader("data/steve_jobs.txt")
loaded_document = loader.load()

In [5]:
loaded_document

[Document(metadata={'source': 'data/steve_jobs.txt'}, page_content='Steven Paul Jobs (February 24, 1955 – October 5, 2011) was an American businessman, inventor, and investor best known for co-founding the technology company Apple Inc. Jobs was also the founder of NeXT and chairman and majority shareholder of Pixar. He was a pioneer of the personal computer revolution of the 1970s and 1980s, along with his early business partner and fellow Apple co-founder Steve Wozniak.\n\nJobs was born in San Francisco in 1955 and adopted shortly afterwards. He attended Reed College in 1972 before withdrawing that same year. In 1974, he traveled through India, seeking enlightenment before later studying Zen Buddhism. He and Wozniak co-founded Apple in 1976 to further develop and sell Wozniak\'s Apple I personal computer. Together, the duo gained fame and wealth a year later with production and sale of the Apple II, one of the first highly successful mass-produced microcomputers.\n\nJobs saw the comme

In [6]:
# chunking the document
text_splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=0
)
chunks_of_text = text_splitter.split_documents(loaded_document)

In [7]:
chunks_of_text

[Document(metadata={'source': 'data/steve_jobs.txt'}, page_content="Steven Paul Jobs (February 24, 1955 – October 5, 2011) was an American businessman, inventor, and investor best known for co-founding the technology company Apple Inc. Jobs was also the founder of NeXT and chairman and majority shareholder of Pixar. He was a pioneer of the personal computer revolution of the 1970s and 1980s, along with his early business partner and fellow Apple co-founder Steve Wozniak.\n\nJobs was born in San Francisco in 1955 and adopted shortly afterwards. He attended Reed College in 1972 before withdrawing that same year. In 1974, he traveled through India, seeking enlightenment before later studying Zen Buddhism. He and Wozniak co-founded Apple in 1976 to further develop and sell Wozniak's Apple I personal computer. Together, the duo gained fame and wealth a year later with production and sale of the Apple II, one of the first highly successful mass-produced microcomputers.\n\nJobs saw the commer

In [8]:
len(chunks_of_text)

55

In [9]:
# create embeddings
embeddings = OpenAIEmbeddings() 

# create vector store
vector_db = FAISS.from_documents(chunks_of_text,embeddings)

In [10]:
vector_db

In [11]:
# retrieve 3 most similar chunks
retriever = vector_db.as_retriever(search_kwargs={"k": 3})

In [ ]:
response = retriever.invoke("Where was Steve Jobs staying, before he died?")
response

[Document(metadata={'source': 'data/steve_jobs.txt'}, page_content='After just one semester, Jobs dropped out of Reed College without telling his parents. Jobs later explained this was because he did not want to spend his parents\' money on an education that seemed meaningless to him. He continued to attend by auditing his classes, including a course on calligraphy that was taught by Robert Palladino. In a 2005 commencement speech at Stanford University, Jobs stated that during this period, he slept on the floor in friends\' dorm rooms, returned Coke bottles for food money, and got weekly free meals at the local Hare Krishna temple. In that same speech, Jobs said: "If I had never dropped in on that single calligraphy course in college, the Mac would have never had multiple typefaces or proportionally spaced fonts".\n\n\n1974–1985:\nPre-Apple:\nIn February 1974, Jobs returned to his parents\' home in Los Altos and began looking for a job. He was soon hired by Atari, Inc. in Los Gatos, C

## Simple use of LCEL (Langchain Expression Language)

In [15]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
# create a prompt template
template = """Answer the question based only on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

# llm
model = ChatOpenAI()

In [ ]:
# format the documents
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# create a chain
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

In [ ]:
response = chain.invoke("Where was Steve Jobs staying, before he died?")
response

'Steve Jobs was staying at his home in Palo Alto, California before he died.'

In [19]:
type(response)

str